In [112]:
import jax
import jax.numpy as jnp
import jax.tree_util as tree
import jraph
import flax
import haiku as hk
import optax
import pickle
import numpy as np
import pandas as pd
import networkx as nx

from pandas.api.types import CategoricalDtype
from typing import Any, Callable, Dict, List, Optional, Tuple


In [281]:
import plotly.express as px
import plotly.graph_objects as go

In [16]:
pd.set_option('display.max_rows', 125)
pd.set_option('display.max_columns', 200)

In [2]:
filename = '~/Downloads/2023-03-26_sendage.h5'
boulders = pd.read_hdf(filename, 'boulders')
ascents_df = pd.read_hdf(filename, 'ascents')
climbs_df = pd.read_hdf(filename, 'climbs')
climbers_df = pd.read_hdf(filename, 'climbers')

In [3]:
boulders = boulders.drop(136733)
boulders['# sends'] = boulders['# onsights'] + boulders['# redpoint'] + boulders['# flashes']
climbs_df['# sends'] = climbs_df['# onsights'] + climbs_df['# redpoint'] + climbs_df['# flashes']

In [11]:
boulders['V grade'] = 0
filt = boulders['grade'].str[-1]=='+'
boulders.loc[filt, 'V grade'] = boulders.loc[filt, 'grade'].str[1:-1].astype(int) + 0.25
filt = boulders['grade'].str[-1]=='-'
boulders.loc[filt, 'V grade'] = boulders.loc[filt, 'grade'].str[1:-1].astype(int) - 0.25
filt = boulders['V grade']==0
boulders.loc[filt, 'V grade'] = boulders.loc[filt, 'grade'].str[1:].astype(int)
boulders.head()

,slug,name,area_id,type,grade_id,bolts,length,grade,description,# onsights,# redpoint,# flashes,# sends,areas_0_name,areas_1_name,areas_2_name,areas_0_slug,areas_1_slug,areas_2_slug,areas_3_name,areas_3_slug,areas_4_name,areas_4_slug,areas_5_name,areas_5_slug,areas_6_name,areas_6_slug,areas_7_name,areas_7_slug,areas_8_name,areas_8_slug,areas_9_name,areas_9_slug,areas_10_name,areas_11_name,areas_10_slug,areas_11_slug,ratings_min,ratings_max,ratings_mean,ratings_count,stars_isna,connected,V grade
36512,superfly-superfly-grand-wall-boulders-squamish...,Superfly,8353,boulder,29,0,NaN,V4,A Squamish all-time classic with physical move...,22,575,74,671,Squamish,Grand Wall Boulders,Superfly,squamish-bc-canada,grand-wall-boulders-squamish-bc-canada,superfly-grand-wall-boulders-squamish-bc-canada,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,5.0,4.143836,438.0,233.0,True,4.00
36410,easy-in-easy-chair-easy-chair-grand-wall-bould...,Easy In An Easy Chair,7521,boulder,29,0,NaN,V4,Traverse left to right on good slopers. A clas...,27,552,88,667,Squamish,Grand Wall Boulders,Easy Chair,squamish-bc-canada,grand-wall-boulders-squamish-bc-canada,easy-chair-grand-wall-boulders-squamish-bc-canada,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,5.0,4.071584,461.0,206.0,True,4.00
36476,titanic-titanic-north-grand-wall-boulders-squa...,Titanic,8360,boulder,24,0,NaN,V3+,Start standing with opposing edges for your ha...,59,465,133,657,Squamish,Grand Wall Boulders,Titanic North,squamish-bc-canada,grand-wall-boulders-squamish-bc-canada,titanic-north-grand-wall-boulders-squamish-bc-...,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,5.0,3.929577,426.0,231.0,True,3.25
35610,trad-killer-heartbreak-hotel-grand-wall-boulde...,Trad Killer,8351,boulder,28,0,NaN,V4,,37,438,78,553,Squamish,Grand Wall Boulders,Heartbreak Hotel,squamish-bc-canada,grand-wall-boulders-squamish-bc-canada,heartbreak-hotel-grand-wall-boulders-squamish-...,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,5.0,4.149457,368.0,185.0,True,4.00
3049,timeless-titanic-south-grand-wall-boulders-squ...,Timeless,8361,boulder,32,0,NaN,V5-,Start matched on the big shelf and climb up an...,22,414,84,520,Squamish,Grand Wall Boulders,Titanic South,squamish-bc-canada,grand-wall-boulders-squamish-bc-canada,titanic-south-grand-wall-boulders-squamish-bc-...,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,5.0,4.168022,369.0,151.0,True,4.75


In [4]:
bascents = ascents_df[np.isin(ascents_df['climb_id'], boulders.index)]
bascents_dated = bascents[~bascents['date'].isna()]
climbers_, counts = np.unique(bascents_dated['user'], return_counts=True)
boulderers_dated = pd.DataFrame({'name': climbers_, '# sends': counts})

In [58]:
bascents_dated['V grade'] = 0
filt = bascents_dated['grade'].str[-1]=='+'
bascents_dated.loc[filt, 'V grade'] = bascents_dated.loc[filt, 'grade'].str[1:-1].astype(int) + 0.25
filt = bascents_dated['grade'].str[-1]=='-'
bascents_dated.loc[filt, 'V grade'] = bascents_dated.loc[filt, 'grade'].str[1:-1].astype(int) - 0.25
filt = bascents_dated['V grade']==0
bascents_dated.loc[filt, 'V grade'] = bascents_dated.loc[filt, 'grade'].str[1:].astype(int)
bascents_dated.head()

/var/folders/bj/_xdx3pvd5yx3t5gm46xh5b340000gn/T/ipykernel_1377/792156011.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bascents_dated['V grade'] = 0


,climb_id,user,date,grade,style,first-ascent,stars,comments,beta,V grade
0,36512,hucklord,2023-02-12,V4,redpoint,False,5.0,,,4.0
1,36512,rajsoni,2023-01-24,V4,flash,False,NaN,,,4.0
2,36512,maude-johnson,2022-11-01,V4,redpoint,False,5.0,,,4.0
3,36512,bethany-van-hierden,2022-10-20,V4,redpoint,False,NaN,2nd go.,,4.0
4,36512,alessandroroome-sandrin,2022-10-12,V4,redpoint,False,4.0,,,4.0


In [5]:
boulderers_dated.describe()

,# sends
count,1967.000000
mean,73.507372
std,156.674864
min,1.000000
25%,5.000000
50%,20.000000
75%,73.000000
max,1907.000000


In [6]:
boulderers_dated.head()

,name,# sends
0,aaron,23
1,aaron-eden,1
2,aaron-guiles,7
3,aaron-martinez,54
4,aaron-pardy,277


In [7]:
climbers_, counts = np.unique(bascents['user'], return_counts=True)
boulderers_ = pd.DataFrame({'name': climbers_, '# sends': counts})
boulderers_.describe()

,# sends
count,2060.000000
mean,78.809709
std,169.691334
min,1.000000
25%,6.000000
50%,22.000000
75%,78.000000
max,2256.000000


In [218]:
squamish_boulders = boulders[boulders['areas_0_name']=='Squamish']

squamish_boulders.loc[squamish_boulders['areas_1_name']=='The Powerline Boulders', 'areas_1_name'] = 'Powerline Boulders'
squamish_boulders.loc[squamish_boulders['areas_2_name']=='Parking Lot Wall', 'areas_2_name'] = 'Parking Lot'
climb_features = pd.concat([squamish_boulders[['V grade', 'ratings_mean']], 
                            pd.get_dummies(squamish_boulders['areas_1_name']), 
                            pd.get_dummies(squamish_boulders['areas_2_name'])], axis=1)
climb_features.head()

,V grade,ratings_mean,,Apron Boulders,Ashlu Valley,Biker Boulders,Brohm Lake,Cheakamus Canyon,Coho Park,Crumpit Woods,Furry Creek,Garabaldi Estates,Grand Wall Boulders,Murrin Park,North Walls,Paradise Valley,Porteau Cove,Powerline Boulders,Shannon Falls,Shannon Springs,Smoke Bluffs,The Chief,The Courtyard,UFO Boulders,Valley Low,,Action Man,Alfa Boulders,Almost Paradise,Angel's Crest Trail,Animal Magnetism,Black Dyke,Bog Wall,Boulder Gully Boulders,Burgers and Fries,Campground Boulders,Cat Lake,Chek,Classic Rock,Close To Home,Corona Crag,Courtyard Boulders,Crag Boulders,Crystal Hunter,Descent Trail,Drive By,Easy Chair,Elemental Boulders,Enchanted Boulders,Forgotten Wall,Free and easy,Fridge Boulders,Gibb's Cave,Gilligan's Island,God Boulders,Gonzales Creek,Grand Wall,Heartbreak Hotel,Highlander Crag,Highway Robbery,Invincible Area Boulders,Israel's Cave Boulders,It's All in the Hips,Lakeside In The Woods,Lipsmack,Lower Sheriff's Badge Trail,Mantel Madness,Margaritaville,Midsummer's Eve Boulders,Neat and Cool,Nightmare Rock,North Gully Trail,Octagon,Paradise Wall,Parking Lot,Penny Lane,Raging Waters,Rainy Day Boulders,Ride the Lightning,Rock Lady,Rolling Stones,Shannon Falls Wall,Short Stories Boulders,Slhanay Trail,Spirit World,Stairway Zone Boulders,Superfly,Survivor,Swamp Thing,The Apron,The Burial Grounds,The Clean Boulders,The Dark Side,The Farm,The Forest,The Forgotten Boulders,The Jungle,The Laboratory,The Lost World,The Magic Kingdom,The Missing Boulders,The Playground Crag,The Portal,The River Boulders,The Snake Pit,The Talus,Thighmaster,Titanic North,Titanic South,Traverse Wall,Undertow,Upper Sheriff's Badge Trail,Vigilante Boulders,Viper,Wendy World,Zodiac Wall Boulders
36512,4.00,4.143836,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
36410,4.00,4.071584,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
36476,3.25,3.929577,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
35610,4.00,4.149457,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3049,4.75,4.168022,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [219]:
len(climb_features)

3328

In [220]:
# edges
squamish_sends = bascents_dated[np.isin(bascents_dated['climb_id'], squamish_boulders.index)]

cat_type = CategoricalDtype(categories=["onsight", "flash", "redpoint"], ordered=True)
squamish_sends['style'] = squamish_sends['style'].astype(cat_type)
squamish_sends['style_num'] = squamish_sends['style'].cat.codes

squamish_sends.head()

/var/folders/bj/_xdx3pvd5yx3t5gm46xh5b340000gn/T/ipykernel_1377/363442703.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  squamish_sends['style'] = squamish_sends['style'].astype(cat_type)
/var/folders/bj/_xdx3pvd5yx3t5gm46xh5b340000gn/T/ipykernel_1377/363442703.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  squamish_sends['style_num'] = squamish_sends['style'].cat.codes


,climb_id,user,date,grade,style,first-ascent,stars,comments,beta,V grade,style_num
0,36512,hucklord,2023-02-12,V4,redpoint,False,5.0,,,4.0,2
1,36512,rajsoni,2023-01-24,V4,flash,False,NaN,,,4.0,1
2,36512,maude-johnson,2022-11-01,V4,redpoint,False,5.0,,,4.0,2
3,36512,bethany-van-hierden,2022-10-20,V4,redpoint,False,NaN,2nd go.,,4.0,2
4,36512,alessandroroome-sandrin,2022-10-12,V4,redpoint,False,4.0,,,4.0,2


In [221]:
n=2
tmp = squamish_sends.groupby('user')['climb_id'].count()
boulderers_min_n = tmp[tmp>n]
squamish_sends = squamish_sends[np.isin(squamish_sends['user'], boulderers_min_n.index)]
len(squamish_sends)

54981

In [222]:
squamish_sends = squamish_sends.sort_values(['user', 'date']).set_index(['user', 'date'])

In [223]:
climb_features = climb_features.loc[np.isin(climb_features.index, squamish_sends['climb_id'])]
climb_features = pd.concat([pd.Series(np.arange(len(climb_features)), index=climb_features.index), climb_features], axis=1)
climb_features

,0,V grade,ratings_mean,,Apron Boulders,Ashlu Valley,Biker Boulders,Brohm Lake,Cheakamus Canyon,Coho Park,Crumpit Woods,Furry Creek,Garabaldi Estates,Grand Wall Boulders,Murrin Park,North Walls,Paradise Valley,Porteau Cove,Powerline Boulders,Shannon Falls,Shannon Springs,Smoke Bluffs,The Chief,The Courtyard,UFO Boulders,Valley Low,,Action Man,Alfa Boulders,Almost Paradise,Angel's Crest Trail,Animal Magnetism,Black Dyke,Bog Wall,Boulder Gully Boulders,Burgers and Fries,Campground Boulders,Cat Lake,Chek,Classic Rock,Close To Home,Corona Crag,Courtyard Boulders,Crag Boulders,Crystal Hunter,Descent Trail,Drive By,Easy Chair,Elemental Boulders,Enchanted Boulders,Forgotten Wall,Free and easy,Fridge Boulders,Gibb's Cave,Gilligan's Island,God Boulders,Gonzales Creek,Grand Wall,Heartbreak Hotel,Highlander Crag,Highway Robbery,Invincible Area Boulders,Israel's Cave Boulders,It's All in the Hips,Lakeside In The Woods,Lipsmack,Lower Sheriff's Badge Trail,Mantel Madness,Margaritaville,Midsummer's Eve Boulders,Neat and Cool,Nightmare Rock,North Gully Trail,Octagon,Paradise Wall,Parking Lot,Penny Lane,Raging Waters,Rainy Day Boulders,Ride the Lightning,Rock Lady,Rolling Stones,Shannon Falls Wall,Short Stories Boulders,Slhanay Trail,Spirit World,Stairway Zone Boulders,Superfly,Survivor,Swamp Thing,The Apron,The Burial Grounds,The Clean Boulders,The Dark Side,The Farm,The Forest,The Forgotten Boulders,The Jungle,The Laboratory,The Lost World,The Magic Kingdom,The Missing Boulders,The Playground Crag,The Portal,The River Boulders,The Snake Pit,The Talus,Thighmaster,Titanic North,Titanic South,Traverse Wall,Undertow,Upper Sheriff's Badge Trail,Vigilante Boulders,Viper,Wendy World,Zodiac Wall Boulders
36512,0,4.00,4.143836,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
36410,1,4.00,4.071584,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
36476,2,3.25,3.929577,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
35610,3,4.00,4.149457,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3049,4,4.75,4.168022,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140242,2999,6.00,5.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
140240,3000,6.75,5.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
140241,3001,7.00,5.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,

In [224]:
squamish_sends

climb_id grade     style  first-ascent  stars  \
user          date                                                        
aaron         2019-04-21     36410    V4  redpoint         False    4.0   
              2019-04-21      3082    V3  redpoint         False    3.0   
              2021-07-11      3049    V5  redpoint         False    5.0   
aaron-pardy   2021-02-27     36410    V4  redpoint         False    5.0   
              2021-02-27    136621   V0-   onsight         False    2.0   
...                            ...   ...       ...           ...    ...   
zoebeauchemin 2022-10-02    129022    V9  redpoint         False    4.0   
              2023-03-21     36485    V7  redpoint         False    4.0   
              2023-03-21     64154    V5  redpoint         False    3.0   
              2023-03-22     92069    V9  redpoint         False    4.0   
              2023-03-22      3076   V10  redpoint         False    3.0   

                                                                   comments  \
user          date                                                            
aaron         2019-04-21                                                      
              2019-04-21                                                      
              2021-07-11                            Nice climb. Slopey top.   
aaron-pardy   2021-02-27                                                      
              2021-02-27                                                      
...                                                                     ...   
zoebeauchemin 2022-10-02  Gnarly approach for the birks! Fun little prow...   
              2023-03-21                                                      
              2023-03-21                                                      
              2023-03-22                                                      
              2023-03-22                                                      

                         beta  V grade  style_num  
user          date                                 
aaron         2019-04-21          4.00          2  
              2019-04-21          3.00          2  
              2021-07-11          5.00          2  
aaron-pardy   2021-02-27          4.00          2  
              2021-02-27         -0.25          0  
...                       ...      ...        ...  
zoebeauchemin 2022-10-02          9.00          2  
              2023-03-21          7.00          2  
              2023-03-21          5.00          2  
              2023-03-22          9.00          2  
              2023-03-22         10.00          2  

[54981 rows x 9 columns]

In [225]:
last_user, last_date = squamish_sends.index[0]

senders = []
receivers = []
edge_features = []
for i in range(1, len(squamish_sends)):
    user, date = squamish_sends.index[i]
    if user != last_user:
        last_user = user
        last_date = date
        continue
    if date != last_date:
        senders.append(climb_features.loc[squamish_sends.iloc[i-1]['climb_id'], 0.])
        receivers.append(climb_features.loc[squamish_sends.iloc[i]['climb_id'], 0.])
        edge_features.append(np.concatenate([np.squeeze(np.diff(squamish_sends.iloc[i-1:i+1][['V grade', 'stars', 'style_num']], axis=0)),
                                             [(date - last_date)/pd.Timedelta("1 days")]]))
    else:
        same_day = squamish_sends.iloc[:i].loc[[(user, date)]]
        for j, row in same_day.iterrows():
            senders.append(climb_features.loc[row['climb_id'], 0.])
            receivers.append(climb_features.loc[squamish_sends.iloc[i]['climb_id'], 0.])
            edge_features.append(np.concatenate([squamish_sends.iloc[i][['V grade', 'stars', 'style_num']].values 
                                                 - row[['V grade', 'stars', 'style_num']].values, [0.]]))
            senders.append(receivers[-1])
            receivers.append(senders[-2])
            edge_features.append(-edge_features[-1])
    
    last_user = user
    last_date = date
    
len(senders), len(receivers), len(edge_features)

# senders = 
# receivers = 

(305564, 305564, 305564)

In [227]:
edge_features = np.array(edge_features)
edges_df = pd.DataFrame({'senders': senders, 'receivers': receivers, 
                         'delta_v': edge_features[:,0], 'delta_stars': edge_features[:,1], 'delta_style': edge_features[:,2]})

In [234]:
edges_df = edges_df.groupby(['senders', 'receivers']).mean().reset_index().fillna(0)
edges_df

,senders,receivers,delta_v,delta_stars,delta_style
0,0.0,1.0,-0.010949,0.000000,-0.051095
1,0.0,2.0,-1.005882,-0.269231,-0.247059
2,0.0,3.0,-0.008772,-0.074074,0.017544
3,0.0,4.0,0.834615,0.093023,-0.076923
4,0.0,5.0,0.025362,0.000000,0.057971
...,...,...,...,...,...
159580,3002.0,3000.0,-1.250000,0.000000,0.000000
159581,3002.0,3001.0,-1.000000,0.000000,0.000000
159582,3003.0,225.0,-6.750000,0.000000,-1.000000
159583,3003.0,2150.0,-2.750000,-1.000000,0.000000


In [250]:
graph = jraph.GraphsTuple(nodes=jnp.eye(3004),
                          edges=jnp.array(edges_df[['delta_v']].values),
                          receivers=jnp.array(edges_df['receivers'].values).astype(jnp.int32),
                          senders=jnp.array(edges_df['senders'].values).astype(jnp.int32),
                          globals=None,
                          n_node=3004, n_edge=159585
)

In [369]:

def add_self_edges_fn(receivers: jnp.ndarray, senders: jnp.ndarray,
                      total_num_nodes: int) -> Tuple[jnp.ndarray, jnp.ndarray]:
  """Adds self edges. Assumes self edges are not in the graph yet."""
  receivers = jnp.concatenate((receivers, jnp.arange(total_num_nodes)), axis=0)
  senders = jnp.concatenate((senders, jnp.arange(total_num_nodes)), axis=0)
  return receivers, senders


# Adapted from https://github.com/deepmind/jraph/blob/master/jraph/_src/models.py#L506
def GraphConvolution(update_node_fn: Callable,
                     aggregate_nodes_fn: Callable = jax.ops.segment_sum,
                     add_self_edges: bool = False,
                     symmetric_normalization: bool = True) -> Callable:
  """Returns a method that applies a Graph Convolution layer.

  Graph Convolutional layer as in https://arxiv.org/abs/1609.02907,
  NOTE: This implementation does not add an activation after aggregation.
  If you are stacking layers, you may want to add an activation between
  each layer.
  Args:
    update_node_fn: function used to update the nodes. In the paper a single
      layer MLP is used.
    aggregate_nodes_fn: function used to aggregates the sender nodes.
    add_self_edges: whether to add self edges to nodes in the graph as in the
      paper definition of GCN. Defaults to False.
    symmetric_normalization: whether to use symmetric normalization. Defaults to
      True.

  Returns:
    A method that applies a Graph Convolution layer.
  """

  def _ApplyGCN(graph: jraph.GraphsTuple) -> jraph.GraphsTuple:
    """Applies a Graph Convolution layer."""
    nodes, _, receivers, senders, _, _, _ = graph

    # First pass nodes through the node updater.
    nodes = update_node_fn(nodes)
    # Equivalent to jnp.sum(n_node), but jittable
    total_num_nodes = tree.tree_leaves(nodes)[0].shape[0]
    if add_self_edges:
      # We add self edges to the senders and receivers so that each node
      # includes itself in aggregation.
      # In principle, a `GraphsTuple` should partition by n_edge, but in
      # this case it is not required since a GCN is agnostic to whether
      # the `GraphsTuple` is a batch of graphs or a single large graph.
      conv_receivers, conv_senders = add_self_edges_fn(receivers, senders,
                                                       total_num_nodes)
    else:
      conv_senders = senders
      conv_receivers = receivers

    # pylint: disable=g-long-lambda
    if symmetric_normalization:
      # Calculate the normalization values.
      count_edges = lambda x: jax.ops.segment_sum(
          jnp.ones_like(conv_senders), x, total_num_nodes)
      sender_degree = count_edges(conv_senders)
      receiver_degree = count_edges(conv_receivers)

      # Pre normalize by sqrt sender degree.
      # Avoid dividing by 0 by taking maximum of (degree, 1).
      nodes = tree.tree_map(
          lambda x: x * jax.lax.rsqrt(jnp.maximum(sender_degree, 1.0))[:, None],
          nodes,
      )
      # Aggregate the pre-normalized nodes.
      nodes = tree.tree_map(
          lambda x: aggregate_nodes_fn(x[conv_senders], conv_receivers,
                                       total_num_nodes), nodes)
      # Post normalize by sqrt receiver degree.
      # Avoid dividing by 0 by taking maximum of (degree, 1).
      nodes = tree.tree_map(
          lambda x:
          (x * jax.lax.reciprocal(jnp.maximum(receiver_degree, 1.0))[:, None]),
          nodes,
      )
    else:
      nodes = tree.tree_map(
          lambda x: aggregate_nodes_fn(x[conv_senders], conv_receivers,
                                       total_num_nodes), nodes)
    # pylint: enable=g-long-lambda
    return graph._replace(nodes=nodes)

  return _ApplyGCN

In [370]:
def gcn_definition(graph: jraph.GraphsTuple) -> jraph.GraphsTuple:
  """Defines a GCN for regression.
  Args:
    graph: GraphsTuple the network processes.

  Returns:
    output graph with updated node values.
  """
  gn = GraphConvolution(
      update_node_fn=lambda n: jax.nn.relu(hk.Linear(16)(n)),
      add_self_edges=True)
  graph = gn(graph)

  gn = GraphConvolution(
      update_node_fn=hk.Linear(1))
  graph = gn(graph)
  return graph

In [347]:
# GAT implementation adapted from https://github.com/deepmind/jraph/blob/master/jraph/_src/models.py#L442.
def GAT(attention_query_fn: Callable,
        attention_logit_fn: Callable,
        node_update_fn: Optional[Callable] = None,
        add_self_edges: bool = True) -> Callable:
  """Returns a method that applies a Graph Attention Network layer.

  Graph Attention message passing as described in
  https://arxiv.org/pdf/1710.10903.pdf. This model expects node features as a
  jnp.array, may use edge features for computing attention weights, and
  ignore global features. It does not support nests.
  Args:
    attention_query_fn: function that generates attention queries from sender
      node features.
    attention_logit_fn: function that converts attention queries into logits for
      softmax attention.
    node_update_fn: function that updates the aggregated messages. If None, will
      apply leaky relu and concatenate (if using multi-head attention).

  Returns:
    A function that applies a Graph Attention layer.
  """
  # pylint: disable=g-long-lambda
  if node_update_fn is None:
    # By default, apply the leaky relu and then concatenate the heads on the
    # feature axis.
    node_update_fn = lambda x: jnp.reshape(
        jax.nn.leaky_relu(x), (x.shape[0], -1))

  def _ApplyGAT(graph: jraph.GraphsTuple) -> jraph.GraphsTuple:
    """Applies a Graph Attention layer."""
    nodes, edges, receivers, senders, _, _, _ = graph
    # Equivalent to the sum of n_node, but statically known.
    try:
      sum_n_node = nodes.shape[0]
    except IndexError:
      raise IndexError('GAT requires node features')

    # Pass nodes through the attention query function to transform
    # node features, e.g. with an MLP.
    nodes = attention_query_fn(nodes)

    total_num_nodes = tree.tree_leaves(nodes)[0].shape[0]
    if add_self_edges:
      # We add self edges to the senders and receivers so that each node
      # includes itself in aggregation.
      receivers, senders = add_self_edges_fn(receivers, senders,
                                             total_num_nodes)

    # We compute the softmax logits using a function that takes the
    # embedded sender and receiver attributes.
    sent_attributes = nodes[senders]
    received_attributes = nodes[receivers]
    att_softmax_logits = attention_logit_fn(sent_attributes,
                                            received_attributes, edges)

    # Compute the attention softmax weights on the entire tree.
    att_weights = jraph.segment_softmax(
        att_softmax_logits, segment_ids=receivers, num_segments=sum_n_node)

    # Apply attention weights.
    messages = sent_attributes * att_weights
    # Aggregate messages to nodes.
    nodes = jax.ops.segment_sum(messages, receivers, num_segments=sum_n_node)

    # Apply an update function to the aggregated messages.
    nodes = node_update_fn(nodes)

    return graph._replace(nodes=nodes)

  # pylint: enable=g-long-lambda
  return _ApplyGAT

In [318]:
def gat_definition(graph: jraph.GraphsTuple) -> jraph.GraphsTuple:
  """Defines a GAT network for the karate club node classification task.

  Args:
    graph: GraphsTuple the network processes.

  Returns:
    output graph with updated node values.
  """

  def _attention_logit_fn(sender_attr: jnp.ndarray, receiver_attr: jnp.ndarray,
                          edges: jnp.ndarray) -> jnp.ndarray:
    del edges
    x = jnp.concatenate((sender_attr, receiver_attr), axis=1)
    return hk.Linear(1)(x)

  gn = GAT(
      attention_query_fn=lambda n: hk.Linear(4)(n),
      attention_logit_fn=_attention_logit_fn,
      node_update_fn=None,
      add_self_edges=True)
  graph = gn(graph)

  gn = GAT(
      attention_query_fn=lambda n: hk.Linear(4)(n),
      attention_logit_fn=_attention_logit_fn,
      node_update_fn=hk.Linear(1),
      add_self_edges=True)
  graph = gn(graph)
  return graph

In [376]:
def optimize_(graph: jraph.GraphsTuple, grades, network: hk.Transformed, num_steps: int) -> jnp.ndarray:
  """Solves the karate club problem by optimizing the assignments of students."""
  params = network.init(jax.random.PRNGKey(1234), graph)

  @jax.jit
  def predict(params: hk.Params) -> jnp.ndarray:
    decoded_graph = network.apply(params, graph)
    return decoded_graph.nodes.squeeze()

  @jax.jit
  def prediction_loss(params: hk.Params) -> jnp.ndarray:
    decoded_graph = network.apply(params, graph)
    return jnp.mean(jnp.square(grades[:20] - decoded_graph.nodes[:20]))

  opt_init, opt_update = optax.adam(5e-3)
  opt_state = opt_init(params)

  @jax.jit
  def update(params: hk.Params, opt_state) -> Tuple[hk.Params, Any]:
    """Returns updated params and state."""
    g = jax.grad(prediction_loss)(params)
    updates, opt_state = opt_update(g, opt_state)
    return optax.apply_updates(params, updates), opt_state

  @jax.jit
  def accuracy(params: hk.Params) -> jnp.ndarray:
    decoded_graph = network.apply(params, graph)
    return jnp.sqrt(jnp.mean(jnp.square(decoded_graph.nodes - grades)))

  for step in range(num_steps):
    print(f"step {step} error {accuracy(params).item():.2f}")
    params, opt_state = update(params, opt_state)

  return predict(params)

In [377]:
grades = (climb_features['V grade'].values - climb_features['V grade'].mean())/climb_features['V grade'].std()

In [378]:
network = hk.without_apply_rng(hk.transform(gcn_definition))
result = optimize_(graph, jnp.array(grades), 
                   network, num_steps=100)

step 0 error 1.00
step 1 error 1.00
step 2 error 1.00
step 3 error 1.00
step 4 error 1.00
step 5 error 1.00
step 6 error 1.00
step 7 error 1.00
step 8 error 1.00
step 9 error 1.00
step 10 error 1.00
step 11 error 1.00
step 12 error 1.00
step 13 error 1.00
step 14 error 1.00
step 15 error 1.00
step 16 error 1.00
step 17 error 1.00
step 18 error 1.00
step 19 error 1.00
step 20 error 1.00
step 21 error 1.00
step 22 error 1.00
step 23 error 1.00
step 24 error 1.00
step 25 error 1.00
step 26 error 1.00
step 27 error 1.00
step 28 error 1.00
step 29 error 1.00
step 30 error 1.00
step 31 error 1.00
step 32 error 1.00
step 33 error 1.00
step 34 error 1.00
step 35 error 1.00
step 36 error 1.00
step 37 error 1.00
step 38 error 1.00
step 39 error 1.00
step 40 error 1.00
step 41 error 1.00
step 42 error 1.00
step 43 error 1.00
step 44 error 1.00
step 45 error 1.00
step 46 error 1.00
step 47 error 1.00
step 48 error 1.00
step 49 error 1.00
step 50 error 1.00
step 51 error 1.00
step 52 error 1.00
ste

In [380]:
px.scatter(x=grades, y=result)

ValueError: All arguments should have the same length. The length of argument `wide_variable_0` is 1, whereas the length of  previously-processed arguments ['x'] is 3004

In [379]:
result

Array([[-0.0306584 ],
       [-0.02996351],
       [-0.02888955],
       ...,
       [-0.26796678],
       [-0.26796678],
       [-0.34697673]], dtype=float32)